In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import igraph
import time
import gc
from os import path
import ipython_memory_usage
import numpy as np

In [2]:
%ipython_memory_usage_start

'memory profile enabled'

In [2] used 0.1094 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 126.67 MiB


Загрузка данных

In [3]:
vertices = pd.read_csv('data\\vertices.csv') ## вершины ребер, то есть юр.лица
edges = pd.read_csv('data\\edges.csv') ## ребра графа, то есть транзакции между юр.лицами

In [3] used 259.5664 MiB RAM in 3.06s, peaked 16.11 MiB above current, total RAM usage 386.23 MiB


In [4]:
vertices = vertices.fillna(0)

In [4] used 0.1797 MiB RAM in 0.24s, peaked 116.96 MiB above current, total RAM usage 386.41 MiB


Добавим в данные о вершинах информацию о том, представлено ли юр.лицо в тестовой выборке

Закодируем данные о типе компании и ОКВЭД

In [5]:
my_encoder = LabelEncoder()
vertices['company_type']= my_encoder.fit_transform(vertices['company_type'])

In [5] used -3.0781 MiB RAM in 0.32s, peaked 13.10 MiB above current, total RAM usage 383.34 MiB


## Validation of models

1. Выбираем фирмыы, у которых более 1000 транзакций
2. Из этих фирм выбираем случайныые 100 фирм
3. У этих фирм оставляем для тренировочной выборки 500 транзакций
4. Среди всех транзакий оставленных для тестовой выборки выбираем случайные 100 000 транзакций
5. Пытаемся их предсказать

In [6]:
## данные о количестве транзакций для каждой фирмы
comp_counts = edges['id_1'].append(edges['id_2'], ignore_index = True).value_counts()

cv = []
companies_cv = []
cv_length = 3
randoms = [13, 4444, 555]

for i in range(0,cv_length):
    cur_random = randoms[i]
    ## выберем 100 случайных компаний, у которых транзакций более 1_000

##### БЫЛИ ТОЛЬКО КОМПАНИИ С БОЛЕЕ ЧЕМ 1000 ТРАНЗАКЦИЯМИ
    
    sample_comp = comp_counts[comp_counts>=1000].sample(100, random_state = cur_random).index 
    ##Выберем случайные 500 транзакций у выбаранных фирм, чтобы оставить их для обучающей выборки
    sampled_train_edges = pd.DataFrame()
    for company in sample_comp:
        temp_sampled = edges[(edges['id_1'] == company) | (edges['id_2'] == company)].sample(500, random_state = cur_random)
        sampled_train_edges = sampled_train_edges.append(temp_sampled)
    ## выберем 100 000 случайных транзакций среди компаний, у которых более 700 транзакций, предварительно исключив 
    ## данные о транзакциях, оставленных для обучения
    temp_edges = edges[edges.index.isin(sampled_train_edges.index) == False]
    sampled_test_edges = temp_edges[temp_edges['id_1'].isin(sample_comp) | temp_edges['id_2'].isin(sample_comp)].sample(100_000, random_state = cur_random)
    train = edges[edges.index.isin(sampled_test_edges.index) == False].index.values
    test = sampled_test_edges.index.values
    cv.append((train, test))
    companies_cv.append(sample_comp)

In [6] used 355.1172 MiB RAM in 15.21s, peaked 161.73 MiB above current, total RAM usage 738.45 MiB


## Building models

In [7]:
from lightgbm import LGBMClassifier
import catboost as cat
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [7] used 2.8945 MiB RAM in 0.44s, peaked 0.00 MiB above current, total RAM usage 741.35 MiB


Для того, чтобы максимально повторить задание будем строить и оценивть модель следующим образом:\
1. Берем список фирм, для которых возможно удалены транзации(sample_comp)
2. Строим список всех возможных транзакций для нашего списка фирм
3. Удаляем из данного списка те транзакции, которые уже есть в нашей выборке
4. Предсказываем вероятность того, что данная транзакция действительно правдива

Т.к. всего у нас 1534749 фирм, то множество всех уникальных комбинаций 1534749^2. Это слишком много, поэтому будем строить модель отдельно для каждой фирмы. Тогда количество всех комбинаций будет лишь 1534749

In [8]:
vertices['is_true'] = 0
vert_id_istrue = vertices[['id', 'is_true']]
vert_no_true = vertices.drop(['is_true'], axis = 'columns')

In [8] used 144.5547 MiB RAM in 0.22s, peaked 0.00 MiB above current, total RAM usage 885.90 MiB


In [9]:
def custom_cv(model, cat_features):
#    global predicted_prob, train, test, temp, fold_result, X, temp_1, temp_2, y
    
    ## Для кэтбуса упоминание о том, что надо проверрить категориальные фичи
    if type(model).__name__ == 'CatBoostClassifier':
        print('Check catfeatures')
    
    ## Создаем пустой вектор результатов для каждого фолда
    cv_results = []
    for fold in range(0,cv_length):
        
        ## Список всех ребер для тренировки, которые действительно существуют
        train = edges[edges.index.isin(cv[fold][0])][['id_1', 'id_2']]
        ## Список всех ребер для теста, которые действительно существуют
        test = edges[edges.index.isin(cv[fold][1])][['id_1', 'id_2']]
        ## Делаем из тестовой выборки список tuples
        test = [edge[1:] for edge in test.to_records().tolist()]
        
        ## Пустой датафрейм с вероятностями для каждого ребра
        predicted_prob = pd.DataFrame()
        for firm in companies_cv[fold]:
            ## таймер для подсчета времени обучения модели для 1 фирмы
            start = time.time()
            
            ## Создаем список всех фирм, с которой компания имеет связи.
            temp_1 = train[train['id_1'] == firm]['id_2'].rename('id')
            temp_2 = train[train['id_2'] == firm]['id_1'].rename('id')
            temp = pd.DataFrame(temp_1.append(temp_2, ignore_index = True))
            temp['is_true'] = 1
            temp = temp.drop_duplicates(['id', 'is_true'])
            
            ## Добавляем в список все фирмы, с которыми компания могла бы иметь связь
            temp = temp.append(vert_id_istrue, ignore_index = True)
            temp = temp.drop_duplicates(subset = ['id'])
            temp = temp.merge(vert_no_true, on = ['id'], how = 'inner')

            ## Удаляем ненужную инфу из тренировочной выбори
            X = temp.drop(['is_true', 'id'], axis = 'columns')
            
            ## Для кэтбуста делаем кэтфичи
            if cat_features == True:
                X['main_okved'] = X['main_okved'].astype(str)  ## делаем строку
            
            ## target value
            y = temp['is_true']
            
            
            model_trained = model.fit(X, y)
            proba = pd.DataFrame(model_trained.predict_proba(X))
            
            ## Создаем датасет с предсказанными ребрами
            proba['id_1'] = firm
            proba['id_2'] = temp['id'].values
            proba['is_true'] = y.values
            ## Прикрепляем в общий датасет с предсказанными вершинами
            predicted_prob = predicted_prob.append(proba)
            print(firm, fold, time.time() - start)
            
        
        del temp_1, temp_2, temp, X, y, train
        gc.collect()
        
        ## Удаляем вершины, кооторые уже были в тренировочной выборке
        predicted_prob = predicted_prob[predicted_prob['is_true'] != 1]
        
        ## Сортируем вершины по вероятности правдивости
        predicted_prob = predicted_prob.sort_values([1], ascending = False)[['id_1', 'id_2']]
        
        ## Делаем из предсказанных вершин список tuples
        predicted_prob = [edge[1:] for edge in predicted_prob.to_records().tolist()]
        
        ## Выбираем 100_000 самых вероятных вершин, при этом удаляя повторяющиеся
        print('Selecting top 100_000')
        predicted_edges = []
        for edge in predicted_prob:
            if (edge not in predicted_edges) & (tuple([edge[place] for place in [1,0]]) not in predicted_edges) & (len(predicted_edges)<= 99_999):
                predicted_edges.append(edge)
            elif (len(predicted_edges)>99_999):
                break
        ## Подсчитываем, сколько из 100_000 предсказанных вершин действительно есть в тестовой выборке  
        print('scoring')
        score = 0
        
        for predicted_edge in predicted_edges:
            if (predicted_edge in test) | (tuple([predicted_edge[place] for place in [1,0]]) in test):
                score = score + 1
        
        del predicted_prob, test, predicted_edges
        gc.collect()
        
        cv_results.append(score)
        print(score)
        del score
        gc.collect()
    return(cv_results)

In [9] used 0.3047 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 886.21 MiB


In [10]:
def final_results(cv, models_to_use, description, model_cols, cat_features = False):
    names = []
    params = []
    scores = []
    description = [description]*len(models_to_use)
    model_cols = [model_cols]*len(models_to_use)
    
    for model in models_to_use:
        model_score = custom_cv(model, cat_features)
        model_score.append(np.mean(model_score))
        scores.append(model_score)
        
        names.append(type(model).__name__)
        params.append(model.get_params())
        
    final_results = pd.DataFrame({'model_name' : names, 'model_params' : params, 'model_description' : description, 'model_cols' : model_cols})
    
    for cv_split in range(1, np.array(scores).shape[1]+1):
        final_results[str(cv_split)] = np.array(scores).T[cv_split-1]
        
    final_results['mean'] = final_results.iloc[:, -1]
    final_results = final_results.drop([final_results.columns[-2]], axis = 'columns')
    
    if path.exists("final_results.csv"):
        final_results_loaded = pd.read_csv('final_results.csv')
        final_results = final_results_loaded.append(final_results, ignore_index = True)    
    final_results.to_csv('final_results.csv', index = False)

In [10] used 0.0547 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 886.26 MiB


In [11]:
final_results(cv, 
             [LogisticRegression(random_state = 0, n_jobs = -1)],
              'Full graph. As in baseline. Mean, std and sum of n_tran and value for every firm. Filled nans with 0',
             ['main_okved', 'region_code', 'company_type'],
             cat_features = False)

1342335 0 36.3426308631897
1279823 0 33.220423221588135
764716 0 32.2302942276001
1236281 0 33.77120900154114
58408 0 33.826643228530884
271290 0 39.23333263397217
1370326 0 27.6535747051239
996256 0 29.896445751190186
1033582 0 29.341957092285156
747050 0 36.263267040252686
50936 0 34.99074363708496
902371 0 32.822346687316895
660519 0 25.48724675178528
267700 0 32.5036461353302
1378198 0 30.828932285308838
1265607 0 27.906243562698364
1301544 0 36.848222732543945
1263393 0 32.802287578582764
852594 0 30.6029155254364
559269 0 25.9078049659729
1413097 0 24.954498767852783
1381157 0 28.04904294013977
1366516 0 29.54674458503723
341670 0 30.428377389907837
1511197 0 31.667054414749146
1175529 0 26.883264780044556
849853 0 33.346967697143555
531768 0 30.490933895111084
869716 0 32.41748380661011
392563 0 34.07785201072693
1296937 0 20.842031002044678
108285 0 28.88599133491516
1166467 0 35.662713050842285
552547 0 17.0381977558136
109960 0 24.957623958587646
336715 0 22.86548924446106
53

277454 2 30.426280975341797
98841 2 32.09653282165527
393651 2 32.52785396575928
450347 2 26.22710657119751
1350297 2 34.11243033409119
1301544 2 35.20313024520874
531768 2 36.11974334716797
764716 2 32.4731605052948
658550 2 34.39078688621521
1504381 2 41.17266321182251
601217 2 28.22278904914856
1366516 2 33.04849052429199
929264 2 34.1265914440155
1263393 2 40.05502676963806
707356 2 34.357200384140015
1192473 2 35.570581912994385
1370326 2 29.00340747833252
1265607 2 30.94655418395996
835840 2 38.26209902763367
50936 2 34.49790143966675
468918 2 41.82058334350586
1495252 2 25.85738754272461
1378198 2 39.29944705963135
869716 2 38.001837968826294
1342003 2 32.953705072402954
1292407 2 30.869937896728516
1166467 2 33.04780864715576
336715 2 31.681039571762085
1296937 2 25.342493534088135
1203323 2 38.15411448478699
1384741 2 34.60881757736206
559269 2 25.76263165473938
Selecting top 100_000
scoring
992
In [11] used -778.5664 MiB RAM in 12351.61s, peaked 27339.14 MiB above current, to

In [12]:
final_results(cv, 
             [LGBMClassifier(n_estimators = 100, random_state = 0, n_jobs = -1),
              LGBMClassifier(n_estimators = 500, random_state = 0, n_jobs = -1)],
              'Full graph. As in baseline. Mean, std and sum of n_tran and value for every firm. Filled nans with 0',
             ['main_okved', 'region_code', 'company_type'],
             cat_features = False)

1342335 0 7.9886155128479
1279823 0 7.344507455825806
764716 0 6.111635684967041
1236281 0 6.8068013191223145
58408 0 6.9029083251953125
271290 0 5.8772873878479
1370326 0 6.61035418510437
996256 0 6.90952730178833
1033582 0 7.560812711715698
747050 0 6.6352362632751465
50936 0 7.14988374710083
902371 0 7.300509214401245
660519 0 7.319406747817993
267700 0 6.787907123565674
1378198 0 7.5371763706207275
1265607 0 7.401213884353638
1301544 0 6.75992751121521
1263393 0 6.608360052108765
852594 0 8.105301856994629
559269 0 8.065463304519653
1413097 0 7.013224363327026
1381157 0 7.7901999950408936
1366516 0 7.39819598197937
341670 0 7.828071355819702
1511197 0 7.8938963413238525
1175529 0 8.677800178527832
849853 0 7.833061218261719
531768 0 7.843030691146851
869716 0 7.1439008712768555
392563 0 6.940447092056274
1296937 0 8.500274181365967
108285 0 7.97567892074585
1166467 0 7.871690273284912
552547 0 8.463371992111206
109960 0 8.096380710601807
336715 0 8.284939050674438
538092 0 7.529861

559269 2 10.711361408233643
Selecting top 100_000
scoring
1033
1342335 0 34.0733106136322
1279823 0 35.184937715530396
764716 0 28.707252025604248
1236281 0 32.877135276794434
58408 0 22.735246181488037
271290 0 27.067610502243042
1370326 0 33.70389723777771
996256 0 31.690279722213745
1033582 0 36.037657022476196
747050 0 32.17398452758789
50936 0 31.909692764282227
902371 0 36.53236269950867
660519 0 32.91198492050171
267700 0 27.672048330307007
1378198 0 34.10681986808777
1265607 0 35.08719730377197
1301544 0 25.734227418899536
1263393 0 24.747837781906128
852594 0 25.841915369033813
559269 0 37.01008176803589
1413097 0 31.76511001586914
1381157 0 33.88044834136963
1366516 0 30.75774574279785
341670 0 23.74551749229431
1511197 0 34.972506284713745
1175529 0 35.31558799743652
849853 0 35.10913801193237
531768 0 31.34121060371399
869716 0 29.549004554748535
392563 0 27.879467010498047
1296937 0 32.87610721588135
108285 0 34.1556875705719
1166467 0 34.68726658821106
552547 0 34.8169214

835840 2 27.916487455368042
50936 2 37.166510820388794
468918 2 25.950831651687622
1495252 2 37.35998201370239
1378198 2 36.89823007583618
869716 2 29.773494720458984
1342003 2 26.34077739715576
1292407 2 29.50322651863098
1166467 2 36.23901438713074
336715 2 37.921462535858154
1296937 2 35.187859535217285
1203323 2 27.907567739486694
1384741 2 27.544549465179443
559269 2 38.80806612968445
Selecting top 100_000
scoring
9273
In [12] used 2.9766 MiB RAM in 18258.44s, peaked 28547.74 MiB above current, total RAM usage 110.67 MiB


In [13]:
final_results(cv, 
             [CatBoostClassifier(n_estimators = 500, cat_features= ['main_okved', 'region_code', 'company_type'], 
                                 random_state = 0, verbose = False, task_type = 'GPU')],
              'Full graph. As in baseline. Mean, std and sum of n_tran and value for every firm. Filled nans with 0',
             ['main_okved', 'region_code', 'company_type'],
             cat_features = True)

Check catfeatures
1342335 0 95.69275069236755
1279823 0 93.93145799636841
764716 0 93.15487742424011
1236281 0 93.64446949958801
58408 0 90.53352856636047
271290 0 96.64358973503113
1370326 0 96.61669397354126
996256 0 94.10252356529236
1033582 0 94.50410723686218
747050 0 94.89755725860596
50936 0 92.65486598014832
902371 0 93.73049569129944
660519 0 100.39779686927795
267700 0 90.46780276298523
1378198 0 93.28793549537659
1265607 0 89.81654191017151
1301544 0 86.62804889678955
1263393 0 87.41209959983826
852594 0 98.86742806434631
559269 0 97.54542016983032
1413097 0 94.93400001525879
1381157 0 94.56413984298706
1366516 0 94.49764466285706
341670 0 86.77417349815369
1511197 0 95.70355486869812
1175529 0 92.75841283798218
849853 0 90.27644681930542
531768 0 92.36964201927185
869716 0 93.24492454528809
392563 0 89.91245937347412
1296937 0 101.31541657447815
108285 0 93.66443800926208
1166467 0 99.08388590812683
552547 0 97.92681050300598
109960 0 92.91606831550598
336715 0 97.522138595

1342003 2 90.8249819278717
1292407 2 98.05435132980347
1166467 2 98.60030817985535
336715 2 99.04950380325317
1296937 2 106.29179430007935
1203323 2 93.2230453491211
1384741 2 90.68621635437012
559269 2 100.3362238407135
Selecting top 100_000
scoring
21470
In [13] used 6.9102 MiB RAM in 31956.28s, peaked 10837.97 MiB above current, total RAM usage 117.58 MiB


In [3]:
res = pd.read_csv('final_results.csv')

In [9]:
res.to_csv('final_results.csv', index = False)

In [8]:
res

,model_name,model_params,model_description,model_cols,1,2,3,mean,results_leaderboard
0,LogisticRegression,"{'C': 1.0, 'class_weight': None, 'dual': False...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",4351.0,4345.0,4158.0,4284.666667,1126.0
1,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",631.0,1130.0,663.0,808.000000,NaN
2,CatBoostClassifier,"{'iterations': 100, 'verbose': False, 'cat_fea...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",5765.0,5746.0,5724.0,5745.000000,4675.0
3,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",3481.0,3535.0,3266.0,3427.333333,NaN
4,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",19.0,83.0,40.0,47.333333,NaN
5,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",3868.0,4566.0,3822.0,4085.333333,NaN
6,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",5794.0,6073.0,5148.0,5671.666667,2149.0
7,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",2320.0,2199.0,2212.0,2243.666667,NaN
8,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",2343.0,2425.0,2559.0,2442.333333,NaN
9,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",5831.0,5649.0,5351.0,5610.333333,NaN
